# MDLE Assignment 1
## 2. Locality Sensitive Hashing
Students were provided with `covid_news_full.json.bz2` (60357 articles) and `covid_news_small.json.bz2` (18898 articles) files. The task was to implement Locality Sensitive Hashing (LSH) to find similar news articles.
</br></br>
For the following code we have two auxiliary files:
- `lsh_utils.py` which contains the implementation of the functions used by pyspark to implement LSH.
- `constants.py` which contains the constants used in this notebook and the grid search results for the best parameters for the LSH algorithm. It is possible to run the `constants.py` file himself to get the best parameters for the LSH algorithm and to get the best parameters shown in a graph, as shown in `best_b_and_r_expected.png`.

### Imports

In [1]:
import importlib
import pyspark
from itertools import combinations
import json
import math
import time
import os
import shutil
import lsh_utils as lsh
importlib.reload(lsh)

from constants import *

print(BEST_PARAMS)

(13, 11)


### Spark Session

In [2]:
from pyspark.conf import SparkConf

conf = SparkConf()
conf.set("spark.executor.memory", "15g")
conf.set("spark.driver.memory", "15g")

spark = pyspark.SparkContext(appName="Locality Sensitive Hashing", conf=conf).getOrCreate()

# spark = pyspark.SparkContext(appName="Locality Sensitive Hashing").getOrCreate()


24/04/05 22:06:34 WARN Utils: Your hostname, diogo-VivoBook resolves to a loopback address: 127.0.1.1; using 192.168.1.236 instead (on interface wlp1s0)
24/04/05 22:06:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/05 22:06:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2.1
The number of bands and rows should be parameters. Select a combination that finds as
candidates at least 90% of pairs with 85% similarity and less than 5% of pairs with 60% similarity.
</br></br>
The request of the 2.1 question was to implement the LSH algorithm using a combination of parameters that finds as candidates with specific similarity values. For that we had to do some hyperparameter tuning to find the best combination of parameters before executing the algorithm.
After analysing the hyperparameters, using the grid search present in `constants.py`, we have chosen the values of `b=13` and `r=11` since they perform well, meet the requirements requested in the assignment and are the combination of values that will generate the least number of hash functions, improving the performance of the algorithm. There are hyperparameters that perform better, but they generate a large number of hash functions, which can impair the performance of the algorithm.
</br></br>
Some more constants were necessary for the implementation of the LSH algorithm and this ones were thinked and defined by hand. The constants are:
- `SHINGLE_SIZE` which is the size of the shingles that will be used to represent the articles. We have chosen the value of 9, since a lower value would generate a lot of collisions, the value is the mostly used in practice for this kind of problem and since higher values would not have a significant impact on the algorithm's performance.
- `BUCKET_SIZE` which is the size of the buckets that will be used to check if there are collisions in the hash functions generated by the LSH algorithm. We have chosen the value of 100000, since it does not impact the algorithm's time complexity and high values have great impact on the algorithm's performance since wrong collisions will not be as much generated.
- `N_SHINGLES` which was used in the minhash algorithm to generate the signatures of the articles. This is not the real number of shingles generated by the documents, but it's a number that should be greater than it. We have chosen a value of 2**32, since the shingles are converted to their hash representation and this value is the maximum value that can be represented by a 32-bit integer.
- `PRIME` which is the prime number used in the hash functions generated by the LSH algorithm. Since it had, ideally, to be higher than the number of shingles generated by the documents, we have chosen the next prime number after the value of `N_SHINGLES`, which is 4294967311.
- `AB` which is a list of random values used in the hash functions generated by the LSH algorithm. They are generated in execution time, using a custom seed, but are stored in the variables, since it is not suposed to change during the execution of the algorithm and they might be needed in the future for the addition of new documents.
</br></br>
As shown in the next cells, our LSH algorithm consisted of the following steps:
1. Read the data from the file and retrieve the shingles from the articles. We made some text processing to remove ponctuation and convert the text to lowercase.
2. Convert from shingles to a signature representation generated using the minhash algorithm.
3. Convert from signatures to buckets and store them for later use.

### Data Processing and Shingling
Read the data from the file and retrieve the shingles from the articles.

In [3]:
# Read json file and store as tuple
data_shingles = spark.textFile(INPUT_FILE).map(json.loads)\
                .map(lsh.shingles).cache()
                # 1 - Read the file
                # 2 - parse it to json
                # 3 - Convert the json to tuples and convert the text to shingles


### Signatures
Convert from shingles to a signature representation generated using the minhash algorithm.

In [4]:
# Get signatures
data_signatures = data_shingles.map(lsh.min_hash)
                    # 1 - Convert the shingles to minhash signatures


### LSH
Convert from signatures to buckets and store them for later use.

In [5]:
# Split the signatures into N_ROWS elements, put into buckets
data_buckets = data_signatures.map(lsh.signatures_to_buckets)\
                .flatMap(lambda x: [((bucket, i), [x[0]]) for i, bucket in enumerate(x[1])])\
                .reduceByKey(lambda x, y: x + y)
                # 1 - Convert the signatures to buckets
                # 2 - Split the signatures keeping the bucket number (bucket, index of signature)
                # 3 - Join the ids by bucket and index

# We now have a list of buckets with the ids of the signatures that belong to it, if a bucket contains more than one id, it means that those are candidate pairs

# delete the output folder if it exists
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)

# Save the buckets to a file
data_buckets.saveAsTextFile(OUTPUT_FOLDER)


The output folder present in the ex2, represent the output of the LSH algorithm using the full dataset.

## 2.2
Implement a function that, given a news article, returns all other news articles that are at least
85% similar. You should make use of a pre-processed set of candidate pairs, obtained by LSH,
and return only the ones that have Jaccard similarity–considering the shingles–above 85%.
</br></br>
For this question we had to implement a function that, given a news article, returns all other news articles that are at least 85% similar using the Jacard similarity.
To get the candidates, we used the previous processed buckets, stored in the `OUTPUT_FOLDER`, got the distinct candidate pairs, since even if multiple buckets were similar we only wanted to check the similarity between the articles once, generated the list of unique candidates, and then we calculated the Jacard similarity between the articles parallelizing the process using pyspark.

In [ ]:
# Load the data from the lsh
data_buckets = spark.textFile(OUTPUT_FOLDER).map(eval)

In [7]:
def similar_articles(doc_id, expected_similatiry):
    # If the program runs all at once, the data_shingles will be cached and it will be faster to lookup the shingles
    doc_shingles = data_shingles.lookup(doc_id)[0]

    # Candidate pairs of a specific document
    candidate_pairs = data_buckets.filter(lambda x: len(x[1]) > 1)\
                            .map(lsh.get_candidates)\
                            .flatMap(lambda x: x)\
                            .map(lambda x: (x[0], x[1]) if x[0] < x[1] else (x[1], x[0]))\
                            .distinct()\
                            .filter(lambda x: doc_id in x)
                            # 1 - Get the buckets with more than 1 element
                            # 2 - Get the candidates (pairs of similar documents)
                            # 3 - Flatten the list of candidates
                            # 4 - Sort the candidates to avoid duplicates in different order
                            # 5 - Remove duplicates
                            # 6 - Filter the candidates that contain the doc_id

    # List of IDs of the candidates that should be compared to the specific document
    candidates = candidate_pairs.flatMap(lambda x: [x[0], x[1]])\
                                .distinct()\
                                .filter(lambda x: x != doc_id)\
                                .collect()

    return spark.parallelize([(doc_id, x[0], lsh.jaccard_similarity(doc_shingles, x[1])) for _, x in enumerate(data_shingles.filter(lambda x: x[0] in candidates).collect())]).filter(lambda x: x[2] >= expected_similatiry)


doc_similar_articles = similar_articles('1349669108229533696', 0.85)
                              
for i, occurence in enumerate(doc_similar_articles.collect()):
    print(f"Similar article {i}: {occurence}")


Similar article 0: ('1349669108229533696', '1349654208337862656', 1.0)
Similar article 1: ('1349669108229533696', '1349454372560920578', 1.0)
Similar article 2: ('1349669108229533696', '1349823098623819784', 0.9942122186495177)


## 2.3
Using a sample of the dataset, evaluate the LSH method by calculating the Jaccard similarities
and obtaining the percentage of false positives and false negatives.
>**Note**: We average over multiple samples to get more robust values.


For this question we had to evaluate the LSH method by calculating the Jacard similarities and obtaining the percentage of false positives and false negatives. For this we got a sample of the dataset, generated all the combinations of articles in that sample, and calculated the Jacard similarity between them using the shingles. It is important to refer that we considered a document to be similar to another if the Jacard similarity was greater than 85% and not similar otherwise. After getting the similarities between documents, we checked if the same classification was made by the LSH algorithm, and calculated the percentage of false positives and false negatives. All this was done using pyspark to parallelize the process and speed up the calculations.

In [9]:
# For the following code, we assume that if the similarity is > 85% the document is positive and <= 85% the document is negative

# Calculate the false positives, false negatives, true positives and true negatives
def process_combination(comb):
    shingle1, shingle2 = comb
    if shingle1[0] < shingle2[0]:
        comb = (shingle1, shingle2)
    else:
        comb = (shingle2, shingle1)

    similarity = lsh.jaccard_similarity(comb[0][1], comb[1][1])

    if similarity > 0.85:
        if (comb[0][0], comb[1][0]) in candidate_pairs.value:
            true_positives.add(1)
        else:
            false_negatives.add(1)
    else:
        if (comb[0][0], comb[1][0]) in candidate_pairs.value:
            false_positives.add(1)
        else:
            true_negatives.add(1)
            
# Initialize paralell counters
false_positives = spark.accumulator(0)
false_negatives = spark.accumulator(0)
true_positives = spark.accumulator(0)
true_negatives = spark.accumulator(0)

# Get a x sample of data
data_quant = 1000
data_sample = data_shingles.takeSample(withReplacement=False, num=data_quant, seed=0)

# Get the ids of the sample
data_sample_ids = set([x[0] for x in data_sample])

# Create a RDD with the sample
data_sample = spark.parallelize(data_sample)

# Get the candidate pairs
candidate_pairs = set(data_buckets.filter(lambda x: len(x[1]) > 1)\
                    .map(lsh.get_candidates)\
                    .flatMap(lambda x: x)\
                    .map(lambda x: (x[0], x[1]) if x[0] < x[1] else (x[1], x[0]))\
                    .distinct()\
                    .filter(lambda x: x[0] in data_sample_ids and x[1] in data_sample_ids)\
                    .collect())

# Broadcast the candidate pairs to all nodes
candidate_pairs = spark.broadcast(candidate_pairs)

# Generate combinations and process each
data_combinations = data_sample.cartesian(data_sample).filter(lambda x: x[0][0] < x[1][0])
data_combinations.foreach(process_combination)

# Calculate the percentages of false positives and false negatives
false_positives_percentage = false_positives.value / (false_positives.value + true_negatives.value)
false_negatives_percentage = false_negatives.value / (false_negatives.value + true_positives.value)
        
print(f"False positives: {round(false_positives_percentage*100, 2)}%")
print(f"False negatives: {round(false_negatives_percentage*100, 2)}%")

print()
print("True Positives:", true_positives.value)
print("False Negatives:", false_negatives.value)
print("False Positives:", false_positives.value)
print("True Negatives:", true_negatives.value)


24/04/05 22:38:11 WARN TaskSetManager: Stage 17 contains a task of very large size (1546 KiB). The maximum recommended task size is 1000 KiB.


False positives: 0.03%
False negatives: 0.0%

True Positives: 1779
False Negatives: 0
False Positives: 133
True Negatives: 497588


## Delete the Spark Session

In [10]:
# Stop the spark context
spark.stop()